In [ ]:
using Pkg; Pkg.activate("..")
using Plots; pyplot()
using Reproduce
using FileIO
using Statistics
using TOML
using ProgressMeter


data_home = "../oneStepErrors"

In [ ]:
Plots.scalefontsizes(1.5)

In [ ]:
# Get the stupid pyplot warnings out of the way
p=plot(rand(100))
plot(p, size=(100,100))

In [ ]:
# ==================
# --- DATA UTILS ---
# ==================

loadData(itm) = FileIO.load(joinpath(itm.folder_str,"results.jld2"))["results"]
TrainingNRMSE(itm) = loadData(itm)["TrainingNRMSE"]
ValidationNRMSE(itm) = loadData(itm)["ValidationNRMSE"]
TestNRMSE(itm) = loadData(itm)["TestNRMSE"]

function avgOverRuns(ic, getData::Function; runsKey = "seed", aggregate_func=mean, window=1)

    diff_dict = diff(ic)
    @assert length(collect(keys(diff_dict))) == 1
    @assert collect(keys(diff_dict))[1] == runsKey
    
    itms = ic.items
    res = Vector{Float64}[]
    for (itm_idx, itm) ∈ enumerate(itms)
        d = getData(itm)
        
        if window > 1
            d = begin
                
                sm = Float64[]
                for i=1:window:length(d)-window
                    push!(sm, mean(d[i:i+window]))
                end
                sm
            end
        end
        push!(res, d)
    end
    
    vals = zeros(length(res[1]),length(res))
    for i=1:length(res)
        vals[:,i] .= res[i]
    end
    
    μ = aggregate_func(vals,dims=2)
    σ = std(vals, dims=2, corrected=true)./sqrt(length(itms))
    
    return μ, σ
end

cfgDict(diff, arg_tuple) = Dict(Pair.(keys(diff),arg_tuple))

function get_best_IC(ic::ItemCollection, params::Dict; aggregate_func, window=1)
    sub_ic = search(ic, params)
    
    # get all param settings, ignoring runs
    sub_diff = diff(sub_ic)
    delete!(sub_diff, "seed")
    
    iter = Iterators.product(values(sub_diff)...)
    best = Inf
    bestIC = nothing
    bestData = nothing
    for arg ∈ iter
        ic′ = search(sub_ic, cfgDict(sub_diff, arg))
        μ,_ = avgOverRuns(ic′, ValidationNRMSE;window=window)
        
        μ = aggregate_func(μ[1:end])
        if μ ≤ best
            best = μ
            bestIC = ic′
        end
    end
    @assert best != Inf
    
    return bestIC
end

function get_best_setting(ic::ItemCollection, params::Dict; output_data=TrainingNRMSE, aggregate=mean,window=1)
    ic′ = get_best_IC(ic, params; aggregate_func=aggregate, window=window)
    return avgOverRuns(ic′, output_data; aggregate_func=aggregate, window=window)
end

function get_sensitivity(ic::ItemCollection, param::String; phase = TrainingNRMSE, aggregate=mean)
    params = diff(ic)[param]
    
    μs = Float64[]
    σs = Float64[]
    for p ∈ params
        μ, σ = get_best_setting(ic, Dict(param=>p); output_data=phase, aggregate=aggregate)
        if phase == TrainingNRMSE
            μ = μ[end-100:end]
        end
        push!(μs, mean(μ));
        push!(σs, std(μ, corrected=true)/sqrt(length(μ)))
    end
    return μs, σs
end

In [ ]:
# ==================
# --- PLOT UTILS ---
# ==================

fnt = font("serif",22)
global_args = [
    :tickdir=>:out,
    :legend=>:none,
    :grid=>false,
    :ytickfont => fnt,
    :xtickfont => fnt,
    :titlefont => fnt,
    :legendfont=>fnt,
    :xguidefont=>fnt,
    :yguidefont=>fnt
]

pplot(args...;kwargs...) = plot(args...;kwargs...,global_args...)
pplot!(args...;kwargs...) = plot!(args...;kwargs...,global_args...)


tol_muted = parse.(Colorant, ("#88CCEE", 
        "#44AA99", "#117733", "#332288", "#DDCC77", 
        "#999933","#CC6677", "#882255", "#AA4499", 
        "#DDDDDD","#43A385"))

lccolormap = Dict(
    "GVFN"=>tol_muted[4],
    "RNN"=>tol_muted[6],
    "AuxTasks"=>tol_muted[6]
    #"AuxTasks"=>tol_muted[end]
)
getLCColor(ic::ItemCollection) = lccolormap[ic.items[1].parsed_args["agent"]]
getLCColor(agent::String) = lccolormap[agent]

senscolormap = Dict(
    "GVFN"=>tol_muted[4],
    "RNN"=>tol_muted[6],
    "AuxTasks"=>tol_muted[6]
)
getSensColor(ic::ItemCollection) = senscolormap[ic.items[1].parsed_args["agent"]]
getSensColor(agent::String) = senscolormap[agent]

lclsmap = Dict(
    "GVFN"=>:solid,
    "RNN"=>:solid,
    #"AuxTasks"=>:solid
    "AuxTasks"=>:dash
)
getLCLineStyle(ic::ItemCollection) = lclsmap[ic.items[1].parsed_args["agent"]]
getLCLineStyle(agent::String) = lclsmap[agent]

senslsmap = Dict(
    "GVFN"=>:solid,
    "RNN"=>:solid,
    "AuxTasks"=>:dash
)
getSensLineStyle(ic::ItemCollection) = senslsmap[ic.items[1].parsed_args["agent"]]
getSensLineStyle(agent::String) = senslsmap[agent]


struct PlotSpec
    ic::ItemCollection
    model::String
    key::String
    smoothing_window::Int
end

function plot_best_settings(spec::PlotSpec)
    ic = spec.ic
    d = diff(ic)
    
    model = spec.model
    τkey = spec.key
    
    p=pplot()
    for τ ∈ diff(ic)[τkey]
        μ,σ = get_best_setting(ic, Dict(τkey=>τ))
        
        if spec.smoothing_window > 1
            μ, σ = begin
                
                sm = Float64[]
                smσ = Float64[]
                window=spec.smoothing_window
                for i=1:window:length(μ)-window
                    push!(sm, mean(μ[i:i+window]))
                    push!(smσ, mean(σ[i:i+window]))
                end
                sm, smσ
            end
        end
        
        xvals = collect(1:length(μ))
        pplot!(p, xvals, μ, ribbon=σ; label="$(model) τ=$τ", ylabel="NRMSE", xlabel="Steps", lw=3)
    end
    return p
end

function compare_best_settings(spec1::PlotSpec, spec2::PlotSpec)
    ic1, ic2 = spec1.ic, spec2.ic
    model1, model2 = spec1.model, spec2.model
    τkey1, τkey2 = spec1.key, spec2.key
    
    @assert length(diff(ic1)[τkey1]) == length(diff(ic2)[τkey2])
    @assert spec1.smoothing_window == spec2.smoothing_window
    
    plots = []
    for τ ∈ diff(ic1)[τkey1]
        
        p = compare_best_setting([ic1, ic2], [τkey1, τkey2], τ)
        push!(plots, p)
    end
    return plots
end


function compare_best_setting(ics, τkeys, τ; window=1)
    p = pplot()
    model(ic) = ic.items[1].parsed_args["agent"]

    for i=1:length(ics)
        ic, τkey = ics[i], τkeys[i]
        μ,σ = get_best_setting(ic, Dict(τkey=>τ); window=window)
        
        xvals = collect(1:length(μ))
        modelName = model(ic)
        pplot!(p, xvals, μ, ribbon=σ; label="$(modelName) τ=$τ", linestyle=getLCLineStyle(ic),color=getLCColor(ic), lw=2, title="Truncation: $(τ)")
    end
    return p
end

function plot_sensitivity!(p, spec::PlotSpec; phase=TrainingNRMSE, aggregate=mean)
    ic, model, param = spec.ic, spec.model, spec.key
    
    xlabel = endswith(param, "tau") ? "Truncation (p)" : param
    μ, σ = get_sensitivity(ic, param; phase=phase, aggregate=aggregate)
    pplot!(p, diff(ic)[param], μ, errorbar=σ; label="$(model)", linestyle=getSensLineStyle(ic), color=getSensColor(spec.model), lw=3)
end
plot_sensitivity(spec::PlotSpec; phase=TrainingNRMSE, aggregate=mean) = 
    plot_sensitivity!(pplot(), spec; phase=phase, aggregate=aggregate)

function plot_sensitivities(specs::Vector{PlotSpec}; phase=TrainingNRMSE, aggregate=mean)
    p=pplot()
    for i=1:length(specs)
       plot_sensitivity!(p, specs[i]; phase=phase, aggregate=aggregate) 
    end
    p
end

---
# MackeyGlass
---

In [ ]:
RNNSpec(ic, param; window=1) = PlotSpec(ic, "RNN", param, window)
AuxSpec(ic, param; window=1) = PlotSpec(ic, "AuxTasks", param, window)
GVFNSpec(ic, param; window=1) = PlotSpec(ic, "GVFN", param, window)

RNNSpec(ic; window=1) = RNNSpec(ic, "rnn_tau"; window=window)
AuxSpec(ic; window=1) = AuxSpec(ic, "rnn_tau"; window=window)
GVFNSpec(ic; window=1) = GVFNSpec(ic, "gvfn_tau"; window=window)

lc_settings = [
    :ylim=>[0.15,0.7],
]
sens_settings = [
    :ylim=>[0.15,0.7],
]

In [ ]:
gvfn_ic = ItemCollection(joinpath(data_home, "final_runs/mackeyglass_gvfn_final_NRMSE/data"))
diff(gvfn_ic)

In [ ]:
rnn_ic = ItemCollection(joinpath(data_home,"final_runs/mackeyglass_rnn_final_NRMSE/data"));
diff(rnn_ic)

In [ ]:
aux_ic = ItemCollection(joinpath(data_home,"final_runs/mackeyglass_aux_final_NRMSE/data"));
diff(aux_ic)

In [ ]:
p = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic),AuxSpec(aux_ic)]), xscale=:log2)
savefig(p, "mg_tau_sens_train.pdf")
p2 = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic), AuxSpec(aux_ic)]; phase=TestNRMSE), xscale=:log2)
savefig(p2, "mg_tau_sens_test.pdf")
display(p); display(p2)
#pout = plot(p,p2,layout=(1,2),size=(700,400))
#savefig(pout, "mg_tau_sens.pdf")

In [ ]:
p = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic),AuxSpec(aux_ic)];aggregate=median), xscale=:log2)
savefig(p, "mg_tau_sens_median_train.pdf")
p2 = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic), AuxSpec(aux_ic)]; phase=TestNRMSE, aggregate=median), xscale=:log2)
savefig(p2, "mg_tau_sens_median_test.pdf")
display(p); display(p2)
#pout = plot(p,p2,layout=(1,2),size=(700,400))
#savefig(pout, "mg_tau_sens.pdf")

In [ ]:
plots = []
for τ∈[1,8,32]
    push!(plots,compare_best_setting([gvfn_ic, rnn_ic, aux_ic], ["gvfn_tau", "rnn_tau","rnn_tau"],τ; window=1000))
end
pout=pplot(plots..., size=[1500,500]; lc_settings..., layout=(1,3),)
savefig(pout, "mg_lc.pdf")
display(pout)

In [ ]:
plots = []
for τ∈[1,]
    push!(plots,compare_best_setting([gvfn_ic, rnn_ic], ["gvfn_tau", "rnn_tau"],τ; window=1000))
end
pout=plot(plots[1],ylim=[0.15,1.5], title="")
savefig(pout, "mg_lc_tau1.pdf")
display(pout)

In [ ]:
plots = []
for τ∈[2,4]
    push!(plots,compare_best_setting([gvfn_ic, rnn_ic], ["gvfn_tau", "rnn_tau"],τ; window=1000))
end
pout=pplot(plots..., size=[1500,500]; lc_settings..., layout=(1,2))
savefig(pout, "mg_example_lc.pdf")
display(pout)

In [ ]:
gvfn_ic = ItemCollection(joinpath(data_home, "mackeyglass_gvfn_NRMSE/data"))
rnn_ic = ItemCollection(joinpath(data_home,"mackeyglass_rnn_NRMSE/data"));

In [ ]:
gvfn_ss_plots = []
model_ss_plots = []
for τ∈diff(gvfn_ic)["gvfn_tau"]
    subic = search(gvfn_ic, Dict("gvfn_tau"=>τ))
    
    p=plot_sensitivity(GVFNSpec(subic, "gvfn_stepsize"))
    p=pplot(p, title="Truncation: $τ")
    push!(gvfn_ss_plots,p)
    
    p=plot_sensitivity(GVFNSpec(subic, "model_stepsize"))
    p=pplot(p, title="Truncation: $τ")
    push!(model_ss_plots,p)
end
pout = pplot(gvfn_ss_plots..., size=[1500,700],xscale=:log2,)
pout2 = pplot(model_ss_plots..., size=[1500,700],xscale=:log2) 
savefig(pout, "mg_gvfn_gvfnstepsize_sens.pdf")
savefig(pout2, "mg_gvfn_modelstepsize_sens.pdf")
display(pout);display(pout2);

In [ ]:
model_ss_plots = []
for τ∈diff(rnn_ic)["rnn_tau"]
    subic = search(rnn_ic, Dict("rnn_tau"=>τ))
    
    p=plot_sensitivity(RNNSpec(subic, "rnn_lr"))
    p=plot(p, title="Truncation: $τ")
    push!(model_ss_plots,p)
end
pout = pplot(model_ss_plots..., size=[1500,800],xscale=:log2)
savefig(pout, "mg_rnn_sens.pdf")
display(pout)

---
# ACEA 
---

In [ ]:
RNNSpec(ic; window=1) = PlotSpec(ic, "RNN", "rnn_tau", window)
GVFNSpec(ic; window=1) = PlotSpec(ic, "GVFN", "gvfn_tau", window)
AuxSpec(ic; window=1) = PlotSpec(ic, "AuxTasks", "rnn_tau", window)

lc_settings = [
    :ylim=>[0.4,1.3],
]

In [ ]:
gvfn_ic = ItemCollection(joinpath(data_home,"final_runs/acea_gvfn_final_NRMSE/data"))
diff(gvfn_ic)

In [ ]:
rnn_ic = ItemCollection(joinpath(data_home,"final_runs/acea_rnn_final_NRMSE/data"))
diff(rnn_ic)

In [ ]:
aux_ic = ItemCollection(joinpath(data_home,"final_runs/acea_aux_final_NRMSE/data"))
diff(aux_ic)

In [ ]:
p = plot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic),AuxSpec(aux_ic)]), xscale=:log2)
savefig(p, "acea_tau_sens_train.pdf")
p2 = plot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic), AuxSpec(aux_ic)]; phase=TestNRMSE), xscale=:log2)
savefig(p2, "acea_tau_sens_test.pdf")
display(p); display(p2)
#pout = plot(p,p2,layout=(1,2),size=(700,400))
#savefig(pout, "acea_tau_sens.pdf")

In [ ]:
p = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic),AuxSpec(aux_ic)];aggregate=median), xscale=:log2)
savefig(p, "acea_tau_sens_median_train.pdf")
p2 = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic), AuxSpec(aux_ic)]; phase=TestNRMSE, aggregate=median), xscale=:log2)
savefig(p2, "acea_tau_sens_median_test.pdf")
display(p); display(p2)

In [ ]:
plots = []
for τ∈[1,8,32]
    push!(plots,compare_best_setting([gvfn_ic, rnn_ic, aux_ic], ["gvfn_tau", "rnn_tau","rnn_tau"],τ; window=100))
end
pout = pplot(plots..., size=[1500,500]; lc_settings..., layout=(1,3),)
savefig(pout, "acea_lc.pdf")
display(pout)

In [ ]:
gvfn_ic = ItemCollection(joinpath(data_home,"acea_gvfn_NRMSE/data"))
rnn_ic = ItemCollection(joinpath(data_home,"acea_rnn_NRMSE/data"));

In [ ]:
gvfn_ss_plots = []
model_ss_plots = []
for τ∈diff(gvfn_ic)["gvfn_tau"]
    subic = search(gvfn_ic, Dict("gvfn_tau"=>τ))
    
    p=plot_sensitivity(GVFNSpec(subic, "gvfn_stepsize"))
    p=pplot(p, title="Truncation: $τ")
    push!(gvfn_ss_plots,p)
    
    p=plot_sensitivity(GVFNSpec(subic, "model_stepsize"))
    p=pplot(p, title="Truncation: $τ")
    push!(model_ss_plots,p)
end
pout = pplot(gvfn_ss_plots..., size=[1500,800],xscale=:log2,)
pout2 = pplot(model_ss_plots..., size=[1500,800],xscale=:log2) 
savefig(pout, "acea_gvfn_gvfnstepsize_sens.pdf")
savefig(pout2, "acea_gvfn_modelstepsize_sens.pdf")
display(pout);display(pout2);

In [ ]:
model_ss_plots = []
for τ∈diff(rnn_ic)["rnn_tau"]
    subic = search(rnn_ic, Dict("rnn_tau"=>τ))
    
    p=plot_sensitivity(RNNSpec(subic, "rnn_lr"))
    p=pplot(p, title="Truncation: $τ")
    push!(model_ss_plots,p)
end
pout = pplot(model_ss_plots..., size=[1000,700],xscale=:log2)
savefig(pout, "acea_rnn_sens.pdf")
display(pout)

---
# MSO 
---

In [ ]:
RNNSpec(ic; window=1) = PlotSpec(ic, "RNN", "rnn_tau", window)
AuxSpec(ic; window=1) = PlotSpec(ic, "AuxTasks", "rnn_tau", window)
GVFNSpec(ic; window=1) = PlotSpec(ic, "GVFN", "gvfn_tau", window)

plot_settings = [
    #:size=>(900,700),
    :ylim=>[0.3,1.0],
    #:grid=>false
]

In [ ]:
gvfn_ic = ItemCollection(joinpath(data_home,"final_runs/mso_gvfn_final_NRMSE/data"))
diff(gvfn_ic)

In [ ]:
rnn_ic = ItemCollection(joinpath(data_home,"final_runs/mso_rnn_final_NRMSE/data"))
diff(rnn_ic)

In [ ]:
aux_ic = ItemCollection(joinpath(data_home,"final_runs/mso_aux_final_NRMSE/data"))
diff(aux_ic)

In [ ]:
p = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic),AuxSpec(aux_ic)]),  xscale=:log2)
savefig(p, "mso_tau_sens_train.pdf")
p2 = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic),AuxSpec(aux_ic)]; phase=TestNRMSE), xscale=:log2)
savefig(p2, "mso_tau_sens_test.pdf")
display(p); display(p2)
#pout = plot(p,p2,layout=(1,2),size=(700,400))
#savefig(pout, "mso_tau_sens.pdf")

In [ ]:
p = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic),AuxSpec(aux_ic)];aggregate=median), xscale=:log2)
savefig(p, "mso_tau_sens_median_train.pdf")
p2 = pplot(plot_sensitivities([GVFNSpec(gvfn_ic),RNNSpec(rnn_ic), AuxSpec(aux_ic)]; phase=TestNRMSE, aggregate=median), xscale=:log2)
savefig(p2, "mso_tau_sens_median_test.pdf")
display(p); display(p2)
#pout = plot(p,p2,layout=(1,2),size=(700,400))
#savefig(pout, "mg_tau_sens.pdf")

In [ ]:
plots = []
for τ∈[1, 8, 32]
    push!(plots,compare_best_setting([gvfn_ic, rnn_ic, aux_ic], ["gvfn_tau", "rnn_tau","rnn_tau"],τ;window=1000))
end
pout=pplot(plots..., size=[1500,500]; plot_settings..., layout=(1,3),) 
savefig(pout, "mso_lc.pdf")
display(pout)

In [ ]:
gvfn_ic = ItemCollection(joinpath(data_home,"mso_gvfn_NRMSE/data"))
rnn_ic = ItemCollection(joinpath(data_home,"mso_rnn_NRMSE/data"));

In [ ]:
gvfn_ss_plots = []
model_ss_plots = []
for τ∈diff(gvfn_ic)["gvfn_tau"]
    subic = search(gvfn_ic, Dict("gvfn_tau"=>τ))
    
    p=plot_sensitivity(GVFNSpec(subic, "gvfn_stepsize"))
    p=pplot(p, title="Truncation: $τ")
    push!(gvfn_ss_plots,p)
    
    p=plot_sensitivity(GVFNSpec(subic, "model_stepsize"))
    p=pplot(p, title="Truncation: $τ")
    push!(model_ss_plots,p)
end
pout = pplot(gvfn_ss_plots..., size=[1500,700],xscale=:log2,)
pout2 = pplot(model_ss_plots..., size=[1500,700],xscale=:log2) 
savefig(pout, "mso_gvfn_gvfnstepsize_sens.pdf")
savefig(pout2, "mso_gvfn_modelstepsize_sens.pdf")
display(pout);display(pout2);

In [ ]:
model_ss_plots = []
for τ∈diff(rnn_ic)["rnn_tau"]
    subic = search(rnn_ic, Dict("rnn_tau"=>τ))
    
    p=plot_sensitivity(RNNSpec(subic, "rnn_lr"))
    p=pplot(p, title="Truncation: $τ")
    push!(model_ss_plots,p)
end
pout = pplot(model_ss_plots..., size=[1500,700],xscale=:log2)
savefig(pout, "mso_rnn_sens.pdf")
display(pout)

---

# FINAL SETTINGS

In [ ]:
for d in ["mso","mackeyglass","acea"]
    if !isdir("final_settings/$d")
        mkpath("final_settings/$d")
    end
end

In [ ]:
ic = ItemCollection(joinpath(data_home, "mackeyglass_gvfn_NRMSE/data"))
d = diff(ic)
τs = d["gvfn_tau"]

In [ ]:
function getSettings(ic)
    settings = FileIO.load(joinpath(ic.items[1].folder_str, "settings.jld2"))
    cfg = Dict()
    for k ∈ settings["used_keys"]
        cfg[k] = settings["parsed_args"][k]
    end
    cfg
end

function makeConfig(settings_dict, exp, model)
    delete!(settings_dict, "seed")
    cfg = Dict(
        "static_args"=>settings_dict,
        "sweep_args"=> Dict("seed"=>"1:30"),
        "config"=> Dict(
            "save_dir"=>"$(exp)_$(model)_final",
            "exp_file"=>"experiment/timeseries.jl",
            "exp_module_name" => "TimeSeriesExperiment",
            "exp_func_name" => "main_experiment",
            "arg_iter_type" => "iter"
        )
    )
    cfg
end

function makeFinalConfigs(experiments)
    for exp ∈ experiments
        for model ∈ ["gvfn","rnn","aux"]
            for τ ∈ τs
                outfile = "final_settings/$(exp)/$(model)tau$(τ).toml"
                if !isfile(outfile)
                    ic = ItemCollection(joinpath(data_home, "$(exp)_$(model)_NRMSE/data"))
                    K = model == "aux" ? "rnn" : model
                    bestic = get_best_IC(ic, Dict("$(K)_tau"=>τ))
                    settings_dict = getSettings(bestic)
                    cfg = makeConfig(settings_dict, exp, model)
                    open(outfile,"w") do f
                        TOML.print(f, cfg)
                    end
                end
            end
        end
    end
end
makeFinalConfigs() = makeFinalConfigs(["mackeyglass", "mso", "acea"])


In [ ]:
#makeFinalConfigs()

In [ ]:
using GVFN

In [ ]:
function getTargets(n)
    env = GVFN.MackeyGlass()
    targets = Float64[]
    for i=1:n
        push!(targets, step!(env)[1])
    end
    return targets
end

function getReturn(n, γ)
    targets = getTargets(n)
    G = zeros(length(targets))
    for t=n-1:-1:1
        G[t] = targets[t+1] + γ*G[t+1]
    end
    return G
end

getNormalizedReturn(n, γ) = getReturn(n,γ) * (1.0-γ)

plotTargets!(p, n; kwargs...) = pplot!(p, getTargets(n); kwargs...)
plotTargets(n; kwargs...)  = plotTargets!(pplot(), n; kwargs...)

plotReturn!(p, n, γ; kwargs...) = pplot!(p, getReturn(n,γ); kwargs..., label="γ = $γ")
plotReturn(n, γ; kwargs...) = plotReturn!(pplot(), n, γ; kwargs...)
plotNormalizedReturn!(p, n, γ; kwargs...) = pplot!(p, getNormalizedReturn(n,γ); kwargs..., label="γ = $γ")
plotNormalizedReturn(n, γ; kwargs...) = plotNormalizedReturn!(pplot(), n, γ; kwargs...)


In [ ]:
fnt = font("serif")
global_args = [
    :tickdir=>:out,
    :legend=>:none,
    :grid=>false,
    :ytickfont => fnt,
    :xtickfont => fnt,
    :titlefont => fnt,
    :legendfont=>fnt,
    :xguidefont=>fnt,
    :yguidefont=>fnt
]

n=1000
lw=2
p = plotTargets(n; linewidth=3, size=(700,500), color=:black, xlim=[0,200],ylim=[0.2,1.3],label="Observation")

args = [
    :linewidth=>2.5,
    :linestyle=>:dash,
    :legend=>:best
]
plotNormalizedReturn!(p,n, 0.0; args...)
plotNormalizedReturn!(p,n, 0.75; args...)
plotNormalizedReturn!(p,n, 0.90; args...)
plotNormalizedReturn!(p,n, 0.96; args...)
plot(p, legend=:best, xlabel="Steps")

In [ ]:
savefig(p, "mg_Predictions.png")

In [ ]:
n=1000
lw=2

p=plot()
pplot!(p, getTargets(n)[13:end]; linewidth=3, size=(700,500), color=:black, xlim=[0,200],ylim=[0.2,1.3],label="Observation")

args = [
    :linewidth=>2.5,
    :linestyle=>:dash,
    :legend=>:best
]
plotNormalizedReturn!(p,n, 0.0; args...)
plotNormalizedReturn!(p,n, 0.75; args...)
plotNormalizedReturn!(p,n, 0.90; args...)
plotNormalizedReturn!(p,n, 0.96; args...)
plot(p, legend=:best, xlabel="Steps")

In [ ]:
savefig(p, "mg_Predictions2.png")